In [ ]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        #self.board = np.zeros((grid_size,grid_size))
        #self.position = np.zeros((grid_size,grid_size))
        #self.malus_position = np.zeros((grid_size,grid_size))
        
        self.board_humain = np.zeros((grid_size,grid_size))
        self.board_vampire = np.zeros((grid_size,grid_size))
        self.board_warevolves = np.zeros((grid_size,grid_size))
        
        self.board = {"human": self.board_humain, "vampire": self.board_vampire, "warevolves":self.board_warevolves}

        # coordinate of the vampires
        self.vampire_x = 0
        self.vampire_y = 0
        
        # coordinate of the warevolves
        self.warevolves_x = 1
        self.warevolves_y = 1
        
        self.coordinates ={"vampire": np.array((self.vampire_x, self.vampire_y)), "warevolves": np.array((self.warevolves_x, self.warevolves_y))}

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board_humain>0,:] = 256
        b[self.warevolves_y, self.warevolves_y, 2] = 256
        b[self.vampire_x,self.vampire_y,0]= 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, species, train=True): #train
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        #self.position = np.zeros((self.grid_size, self.grid_size))

        #self.position[0:2,:]= -1
        #self.position[:,0:2] = -1
        #self.position[-2:,:] = -1
        #self.position[:,-2:] = -1

        #self.position[self.x, self.y] = 1
        
        reward = 0
        
        not_go_out_of_map = 10000
        
        if action == 0:
            if self.coordinates[species][0] == self.grid_size-3:
                self.coordinates[species][0] = self.coordinates[species][0]self.coordinates[species][0]-1
                reward -= not_go_out_of_map
            else:
                self.coordinates[species][0] = self.coordinates[species][0] + 1
        elif action == 1:
            if self.coordinates[species][0] == 2:
                self.coordinates[species][0] = self.coordinates[species][0]+1
                reward -= not_go_out_of_map
            else:
                self.coordinates[species][0] = self.coordinates[species][0]-1
        elif action == 2:
            if self.coordinates[species][1] == self.coordinates[species][1] - 3:
                self.coordinates[species][1] = self.coordinates[species][1] - 1
                reward -= not_go_out_of_map
            else:
                self.coordinates[species][1] = self.coordinates[species][1] + 1
        elif action == 3:
            if self.coordinates[species][1] == 2:
                self.coordinates[species][1] =self.coordinates[species][1] self.coordinates[species][1] + 1
                reward -= not_go_out_of_map
            else:
                self.coordinates[species][1] = self.coordinates[species][1] - 1

        # North Est (droite)
        elif action == 4:
            if self.coordinates[species][1] == self.grid_size - 3:
                self.coordinates[species][1] = sself.coordinates[species][1] - 1
                reward -= not_go_out_of_map
            if self.coordinates[species][0] == self.grid_size-3:
                self.coordinates[species][0] = self.coordinates[species][0] - 1
                reward -= not_go_out_of_map
            else:
                self.coordinates[species][0] = self.coordinates[species][0] + 1
                self.coordinates[species][1] = self.coordinates[species][1] + 1

        # North ouest 
        elif action == 5:
            if self.coordinates[species][1] == self.grid_size - 3:
                self.coordinates[species][1] = self.coordinates[species][1] - 1
                reward -= not_go_out_of_map
            if self.coordinates[species][0] == 2:
                self.coordinates[species][0] = self.coordinates[species][0] + 1
                reward -= not_go_out_of_map
            else:
                self.coordinates[species][0] = self.coordinates[species][0] - 1
                self.coordinates[species][1] = self.coordinates[species][1] + 1

        # Sud ouest 
        elif action == 6:
            if self.coordinates[species][1] == 2:
                self.coordinates[species][1] = self.coordinates[species][1] + 1
                reward -= not_go_out_of_map
            if self.coordinates[species][0] == 2:
                self.coordinates[species][0] = self.coordinates[species][0] + 1
                reward -= not_go_out_of_map
            else:
                self.coordinates[species][0] = self.coordinates[species][0] - 1
                self.coordinates[species][1] = self.coordinates[species][1] - 1

        # Sud est        
        elif action == 7:
            if self.coordinates[species][1] == 2:
                reward -= not_go_out_of_map
                self.coordinates[species][1] = self.coordinates[species][1] + 1
            if self.coordinates[species][0] == self.grid_size - 3:
                reward -= not_go_out_of_map
                self.coordinates[species][0] = self.coordinates[species][0] - 1
            else:
                self.coordinates[species][0] = self.coordinates[species][0] + 1
                self.coordinates[species][1] = self.coordinates[species][1] - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        count = self.board[species][self.coordinates[species]]
        
        if(species == "vampire"):
            other_species = "warevolves"
        else:
            other_species = "vampire"
        
        if(self.board["human"][self.coordinates[species]] > 0):
            n_species = self.board[species][self.coordinates[species]
            n_human = self.board["human"][self.coordinates[species]
            
            if(n_species > n_human):
                self.board[species][self.coordinates[species]] += self.board["human"][self.coordinates[species]]
                self.board["human"][self.coordinates[species]] = 0
            else:
                p_win = n_species/(n_human*2)
                threshold = random.uniform(0, 1)
                
                # win
                if(p_win >= threshold):
                    self.board[species][self.coordinates[species]] = np.random.binomial(n_species, p_win)
                    self.board[species][self.coordinates[species]] += np.random.binomial(n_human, p_win)
                    self.board["human"][self.coordinates[species]] = 0
                
                # loose
                else:
                    self.board[species][self.coordinates[species]] = 0
                    self.board["human"][self.coordinates[species]] = np.random.binomial(n_human, 1-p_win)
                    
        
        elif(self.board[other_species][self.coordinates[species]] > 0): 
            n_species = self.board[species][self.coordinates[species]]
            n_other_species = self.board[other_species][self.coordinates[species]]
            
            if(n_species >= 1.5*n_other_species):
                self.board[other_species][self.coordinates[species]] = 0 
            
            elif(1.5*n_species <= n_other_species):  
                self.board[species][self.coordinates[species]] = 0 
            
            elif(n_species <= n_other_species and n_species > n_other_species/1.5):
                p_win = n_species/(n_other_species*2)
                threshold = random.uniform(0, 1)
                
                # win
                if(p_win >= threshold):
                    self.board[species][self.coordinates[species]] = np.random.binomial(n_species, p_win)
                    self.board[other_species][self.coordinates[species]] = 0
                
                # loose
                else:
                    self.board[species][self.coordinates[species]] = 0
                    self.board[other_species][self.coordinates[species]] = np.random.binomial(n_other_species, 1-p_win)
                
            elif(n_species > n_other_species and n_species < 1.5*n_other_species)
                p_win = n_species/n_other_species - 0.5
                threshold = random.uniform(0, 1)
                
                # win
                if(p_win >= threshold):
                    self.board[species][self.coordinates[species]] = np.random.binomial(n_species, p_win)
                    self.board[other_species][self.coordinates[species]] = 0
                
                # loose
                else:
                    self.board[species][self.coordinates[species]] = 0
                    self.board[other_species][self.coordinates[species]] = np.random.binomial(n_other_species, 1-p_win)
                        
        #if train: 
        #    reward = -self.malus_position[self.x, self.y]        
        #self.malus_position[self.x, self.y] += 0.5 
        # 0.5 is the value that optimizes the average score. 
        
        reward = reward + self.board[species][self.coordinates[species] - count
                    
        game_over = self.t > self.max_time
                                    
        state = np.concatenate((#self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board["human"].reshape(self.grid_size, self.grid_size,1),
                                self.board["vampire"].reshape(self.grid_size, self.grid_size,1), #species
                               self.board["warevolves"].reshape(self.grid_size, self.grid_size,1),axis=2) #other species
          
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

            
        self.vampire_x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.vampire_y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        self.warevolves_x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.warevolves_y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        self.coordinates ={"vampire": np.array((self.vampire_x, self.vampire_y)), "warevolves": np.array((self.warevolves_x, self.warevolves_y))}

        self.board_human = np.random.poisson(3,size=self.grid_size**2)*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        self.board_human = bonus.reshape(self.grid_size,self.grid_size)
            
        self.board_human[self.vampire_x, self.vampire_y] = 0
        self.board_human[self.warevolves_x, self.warevolves_y] = 0    
        
        init_vampire =  np.random.poisson(4)
        init_warevolves = np.random.poisson(4)
            
        self.board_vampire = np.zeros((grid_size,grid_size))
        self.board_vampire[self.vampire_x, self.vampire_y] = init_vampire
            
        self.board_warevolves = np.zeros((grid_size,grid_size))
        self.board_warevolves[self.warevolves_x, self.warevolves_y] = init_warevolves
        
        self.board = {"human": bonus, "vampire": self.board_vampire, "warevolves":self.board_warevolves}

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        #self.position = np.zeros((self.grid_size, self.grid_size))
        #self.position[0:2,:]= -1
        #self.position[:,0:2] = -1
        #self.position[-2:,:] = -1
        #self.position[:,-2:] = -1
        #self.board[self.x,self.y] = 0
        #self.t = 0

        state = np.concatenate((#self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board["human"].reshape(self.grid_size, self.grid_size,1),
                                self.board["vampire"].reshape(self.grid_size, self.grid_size,1), #species
                               self.board["warevolves"].reshape(self.grid_size, self.grid_size,1),axis=2) #other species
         
        return state